In [ ]:
import tensorflow as tf 
import pandas as pd
import numpy as np
import scipy.sparse as sp
import csv 

#Location of the data_folder 
data_folder      ="/nvme/drive_1/NTDS_Final/"
users            = pd.read_csv(data_folder+"usersdata.csv",delimiter='\t',header=None).values
filtered_users   = pd.read_csv(data_folder+"filtered_users.csv",delimiter=',').values

** I. Split Relations by Type **

In [ ]:
csv_writers = []
for feature in [1,2,3,4,5,6,7]: 
    csv_writers.append(csv.writer(open(data_folder+"filtering/relations_type_"+\
                                       str(feature)+".csv","wt"),delimiter='\t'))

with open(data_folder+"relations.csv", "rt") as f_in:
    reader             = csv.reader(f_in, delimiter="\t")
    
    for i, line in enumerate(reader):    
        node_1             = line[2]  
        node_2             = line[3]

        csv_writers[int(line[4])-1].writerow(line)
         
        if i%1000 is 0:
            print("\r"+str(i) +" relations processed", sep=' ', end='', flush=True)

**II. Track (user_id,index) Pairs **

In [ ]:
user_idx_dict          = {}
filtered_user_idx_dict = {}

for i,user in enumerate(users):
    user_idx_dict[user[0]]=i

for i,user in enumerate(filtered_users): 
    filtered_user_idx_dict[user[0]]=i #user_idx_dict[user[0]]

** III. Create Adjacency Matrices **

In [ ]:
i = 0 
for feature in [1,2,3,4,5,6,7]:
    with open(data_folder+"filtering/relations_type_"+\
              str(feature)+".csv") as f_in: 
        
        adj     = sp.lil_matrix((filtered_users.shape[0], filtered_users.shape[0]), dtype='i2')    
        reader  = csv.reader(f_in, delimiter="\t")
        
        for line in reader:    
        
            i+=1  
            
            if int(line[3]) in filtered_user_idx_dict and int(line[2]) in filtered_user_idx_dict:
                adj[filtered_user_idx_dict[int(line[3])],filtered_user_idx_dict[int(line[2])]] = 1     
                adj[filtered_user_idx_dict[int(line[2])],filtered_user_idx_dict[int(line[3])]] = 1
        
            if i%1000 is 0:
                print("\r"+str(i) +" relations processed", sep=' ', end='', flush=True)
        
        csr = adj.tocsr()

        sp.save_npz(data_folder+"filtering/adjacency_"+str(feature)+".npz",csr)
    